In [ ]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize

# modulo para cargar archivos en formato MATLAB
from scipy.io import loadmat

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Cargar datos
data = np.loadtxt(os.path.join('/content/gdrive/MyDrive/dataset_2p', 'ecoli.csv'), delimiter=",")


In [ ]:
#CREAR X ; Y
X = data[:, :7]
y = data[:, 7]
#m = y.size

In [ ]:
# Son las x que tiene el dataset
input_layer_size  = 7

# se DEFINE CUANTOS ESTADOS DE SALIDA EXISTEN
num_labels = 8

#Numero de filas o ejemplos
m = y.size

#Numero de x es n

In [ ]:
def sigmoid(z):
    """
    Calcula la sigmoide de z.
    """
    return 1.0 / (1.0 + np.exp(-z))

In [ ]:
def lrCostFunction(theta, X, y, lambda_):
    
    # Inicializa algunos valores utiles
    m = y.size
    
    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)
    
    J = 0
    grad = np.zeros(theta.shape)
    
    h = sigmoid(X.dot(theta.T))
    
    temp = theta
    temp[0] = 0
    
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
    
    grad = (1 / m) * (h - y).dot(X) 
    grad = grad + (lambda_ / m) * temp

    return J, grad

In [ ]:
# valores de prueba para los parámetros theta
theta_t = np.array([-2, -1, 1, 2], dtype=float)

# valores de prueba para las entradas
X_t = np.concatenate([np.ones((5, 1)), np.arange(1, 16).reshape(5, 3, order='F')/10.0], axis=1)

# valores de testeo para las etiquetas
y_t = np.array([1, 0, 1, 0, 1])

# valores de testeo para el parametro de regularizacion
lambda_t = 3

In [ ]:
J, grad = lrCostFunction(theta_t, X_t, y_t, lambda_t)

print('Costo         : {:.6f}'.format(J))
print('Costo esperadot: 2.534819')
print('-----------------------')
print('Gradientes:')
print(' [{:.6f}, {:.6f}, {:.6f}, {:.6f}]'.format(*grad))
print('Gradientes esperados:')
print(' [0.146561, -0.548558, 0.724722, 1.398003]');

Costo         : 2.534819
Costo esperadot: 2.534819
-----------------------
Gradientes:
 [0.146561, -0.548558, 0.724722, 1.398003]
Gradientes esperados:
 [0.146561, -0.548558, 0.724722, 1.398003]


In [ ]:
def oneVsAll(X, y, num_labels, lambda_):
  
    # algunas variables utiles
    m, n = X.shape
    
    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    #for c in np.arange(num_labels): solo cuando y esta al incio
  # c+1 se hace para sabr los n resltados

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 100}
        res = optimize.minimize(lrCostFunction, 
                                initial_theta, 
                                (X, (y == (c+1)), lambda_), 
                                jac=True, 
                                method='BFGS',
                                options=options) 
        
        all_theta[c] = res.x

    return all_theta

In [ ]:
lambda_ = 0.1
all_theta = oneVsAll(X, y, num_labels, lambda_)

In [ ]:
def predictOneVsAll(all_theta, X):

    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p + 1

In [ ]:
print("ONE VS ALL DE ESTADO DE SALUD FETAL QUE VARIA ENTRE 1 Y 3\n1 = Muy Saludable \n2 = Normal \n3 = Sospechoso (Revision)\n")
print(X.shape)
pred = predictOneVsAll(all_theta, X)
#Metrica
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))
XPrueba = X[:].copy()
print(XPrueba.shape)

#Pruebas
print(XPrueba)
XPrueba = np.concatenate((np.ones((m, 1)), XPrueba), axis=1)

p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis = 1)
print(p + 1)

#displayData(X[500:510, :])

ONE VS ALL DE ESTADO DE SALUD FETAL QUE VARIA ENTRE 1 Y 3
1 = Muy Saludable 
2 = Normal 
3 = Sospechoso (Revision)

(336, 7)
Precision del conjuto de entrenamiento: 89.88%
(336, 7)
[[0.49 0.29 0.48 ... 0.56 0.24 0.35]
 [0.07 0.4  0.48 ... 0.54 0.35 0.44]
 [0.56 0.4  0.48 ... 0.49 0.37 0.46]
 ...
 [0.61 0.6  0.48 ... 0.44 0.39 0.38]
 [0.59 0.61 0.48 ... 0.42 0.42 0.37]
 [0.74 0.74 0.48 ... 0.31 0.53 0.52]]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 2 2
 2 2 2 2 2 2 2 2 2 4 2 2 2 2 4 2 2 2 2 2 2 2 2 2 2 2 4 2 2 2 4 2 2 2 2 1 2
 2 4 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 4 2 2 2 2 2 2 3 2 2 3 2
 7 4 4 4 4 4 4 4 4 2 4 2 2 3 4 4 4 2 2 2 4 4 4 2 4 2 4 4 4 2 4 2 2 4 4 1 4
 5 5 5 3 5 3 5 5 5 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 3 3 3 3 3 3 3 3